In [1]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder
from sklearn.impute import KNNImputer

# Classificadores:
from sklearn.neighbors import NearestNeighbors

## 1) Loading Data

In [2]:
df_meteorologia = pd.read_csv("..\Dados\Views\dados_meteorologicos.csv", index_col=0)
df_produto_agricola = pd.read_csv(R"..\Dados\Tabela_final\dados_producao_agricola.csv", index_col=0)
df_municipios_sertao = pd.read_csv(r"..\Dados\Views\municipios_sertao.csv", index_col=0)
df_transporte = pd.read_csv(r"..\Dados\Views\custo_de_transporte.csv", index_col=0)
df_recursos_hidricos = pd.read_csv(r"..\Dados\Views\recursos_hidricos.csv", index_col=0)
df_solo = pd.read_csv(r"..\Dados\Views\solos_municipios.csv", index_col=0)

In [3]:
df_classificador = df_municipios_sertao[["IBGE7", "NOME", "LATITUDE", "LONGITUDE"]].copy()

In [4]:
df_classificador.head()

,IBGE7,NOME,LATITUDE,LONGITUDE
0,2700300,ARAPIRACA,-9.75487,-36.6615
1,2700706,BATALHA,-9.67420,-37.1330
2,2700904,BELO MONTE,-9.82272,-37.2770
3,2701209,CACIMBINHAS,-9.40121,-36.9911
4,2701605,CANAPI,-9.11932,-37.5967


In [5]:
# Merges com o dataframe do classificador:
df_classificador = df_classificador.merge(df_transporte, on="IBGE7", how="left")
df_classificador = df_classificador.merge(df_meteorologia, on="IBGE7", how="left")
df_classificador = df_classificador.merge(df_recursos_hidricos[["IBGE7", "AREA_IRRIGADA_TOT"]], on="IBGE7", how="left")

# Merge do dataframe de produtos agrícolas com o de municípios do sertão:
df_produto_agricola = df_produto_agricola.merge(df_municipios_sertao[["IBGE7", "NOME"]], how="inner", on="NOME")

In [6]:
df_solo.head()

,IBGE7,SOLO,AREA_TOTAL
0,2207959,LAd - Latossolos Amarelos Distroficos,64739.067171
1,2207959,PVAe - Argissolos Vermelho-Amarelos Eutroficos,4865.805456
2,2207959,RQo - Neossolos Quartzarenicos Orticos,24660.252674
3,2207934,PVAe - Argissolos Vermelho-Amarelos Eutroficos,23145.966454
4,2207934,RQo - Neossolos Quartzarenicos Orticos,65225.223645


In [7]:
df_produto_agricola["PRODUTO"].unique()

array(['ALGODAO HERBACEO (EM CAROCO)', 'AMENDOIM (EM CASCA)', 'BANANA ',
       'CASTANHA DE CAJU', 'FAVA (EM GRAO)', 'FEIJAO (EM GRAO)',
       'MANDIOCA', 'MANGA', 'MILHO (EM GRAO)', 'TOMATE',
       'CAFE (EM GRAO) ARABICA', 'CAFE (EM GRAO) TOTAL', 'MELANCIA',
       'MELAO', 'BATATA-DOCE', 'BATATA-INGLESA',
       'CAFE (EM GRAO) CANEPHORA', 'URUCUM ', 'CACAU (EM AMENDOA)',
       'SOJA (EM GRAO)', 'SORGO (EM GRAO)', 'MAMONA ', 'UVA',
       'PIMENTA-DO-REINO', 'TRIGO (EM GRAO)', 'GUARANA '], dtype=object)

In [8]:
# Obtendo o produto, respectivamente, de maior Valor e maior rendimento.
rendimentos_max = df_produto_agricola.groupby(["NOME"]).agg({"REND_MEDIO":"max"}).reset_index()
df_agro = rendimentos_max.merge(df_produto_agricola[["NOME", "IBGE7", "REND_MEDIO", "PRODUTO", "VALOR_PROD"]], on=["NOME", "REND_MEDIO"], how="inner")
df_agro = df_agro.drop_duplicates(["NOME", "REND_MEDIO", "PRODUTO"])
max = df_agro.groupby("NOME")["VALOR_PROD"].idxmax()
df_agro_max = df_agro.loc[max].reset_index(drop=True)
df_agro_max.drop(columns=["NOME"], inplace=True)
df_classificador = df_classificador.merge(df_agro_max, on="IBGE7", how="inner")
df_classificador.drop(columns=["REND_MEDIO", "VALOR_PROD"], inplace=True)
df_classificador.rename(columns={"PRODUTO": "PRODUTO_MAIOR_VALOR"},inplace=True)

In [9]:
df_classificador.head()

,IBGE7,NOME,LATITUDE,LONGITUDE,TRANSPORT_COST,PREC_MED,RED_MED,TEMP_MED,VEL_MED,ALTITUDE,AREA_IRRIGADA_TOT,PRODUTO_MAIOR_VALOR
0,2700300,ARAPIRACA,-9.75487,-36.6615,24042.166072,988.689803,1051.169614,26.157050,1.553386,236.84,327.176525,TOMATE
1,2700706,BATALHA,-9.67420,-37.1330,24257.110578,713.023256,1318.900553,27.754333,1.427155,20.83,0.000000,MELANCIA
2,2700904,BELO MONTE,-9.82272,-37.2770,24226.863743,713.023256,1318.900553,27.754333,1.427155,20.83,0.000000,BATATA-DOCE
3,2701209,CACIMBINHAS,-9.40121,-36.9911,24358.206538,209.280265,1585.450518,26.482644,1.062182,278.01,0.000000,MILHO (EM GRAO)
4,2701605,CANAPI,-9.11932,-37.5967,24734.221802,551.553841,1437.777193,27.047011,2.518969,187.00,0.000000,TOMATE


In [10]:
# Obtendo solo de maior área dentro do município
index_solo_max = df_solo.groupby("IBGE7")["AREA_TOTAL"].idxmax()
df_solo_max = df_solo.loc[index_solo_max]
df_classificador = df_classificador.merge(df_solo_max, how="inner", on="IBGE7")
df_classificador.rename(columns={"SOLO":"SOLO_PREDO"}, inplace=True)

In [11]:
df_classificador.head()

,IBGE7,NOME,LATITUDE,LONGITUDE,TRANSPORT_COST,PREC_MED,RED_MED,TEMP_MED,VEL_MED,ALTITUDE,AREA_IRRIGADA_TOT,PRODUTO_MAIOR_VALOR,SOLO_PREDO,AREA_TOTAL
0,2700300,ARAPIRACA,-9.75487,-36.6615,24042.166072,988.689803,1051.169614,26.157050,1.553386,236.84,327.176525,TOMATE,LAd - Latossolos Amarelos Distroficos,35700.513746
1,2700706,BATALHA,-9.67420,-37.1330,24257.110578,713.023256,1318.900553,27.754333,1.427155,20.83,0.000000,MELANCIA,RLe - Neossolos Litolicos Eutroficos,27053.475931
2,2700904,BELO MONTE,-9.82272,-37.2770,24226.863743,713.023256,1318.900553,27.754333,1.427155,20.83,0.000000,BATATA-DOCE,RLe - Neossolos Litolicos Eutroficos,33004.844277
3,2701209,CACIMBINHAS,-9.40121,-36.9911,24358.206538,209.280265,1585.450518,26.482644,1.062182,278.01,0.000000,MILHO (EM GRAO),SXe - Planossolos Haplicos Eutroficos,26836.700678
4,2701605,CANAPI,-9.11932,-37.5967,24734.221802,551.553841,1437.777193,27.047011,2.518969,187.00,0.000000,TOMATE,SXe - Planossolos Haplicos Eutroficos,32083.122951


## 2) Préprocessamento

In [12]:
X = df_classificador.drop(columns=["IBGE7", "NOME", "PRODUTO_MAIOR_VALOR"])
#y = df_classificador["PRODUTO_MAIOR_VALOR"].values

In [13]:
# Onehot ecoding:
one_hot = OneHotEncoder()
X_one_hot = one_hot.fit_transform(X["SOLO_PREDO"].values.reshape(-1, 1)).toarray()
labels = one_hot.get_feature_names_out()
X[labels] = X_one_hot
X.drop(columns=["SOLO_PREDO"], inplace=True)

In [14]:
colunas_num = [i for i in X.columns if i not in labels]

In [15]:
min_max = MinMaxScaler()
X[colunas_num] = min_max.fit_transform(X[colunas_num])

In [16]:
imputer = KNNImputer(n_neighbors=5)
X_imputado = imputer.fit_transform(X)

## 3) Treinamento do KNN

In [17]:
k_neighburs = 5 
knn = NearestNeighbors(n_neighbors=k_neighburs, n_jobs=-1, metric="cosine")
knn.fit(X_imputado)

NearestNeighbors(metric='cosine', n_jobs=-1)

In [18]:
distance, neighbours_indices = knn.kneighbors(X_imputado[160].reshape(1, -1), n_neighbors=k_neighburs+1)

In [19]:
distancias_corrigidas = distance[0][1:]
neighbours_indices_corrigidos = neighbours_indices[0][1:]

In [20]:
df_classificador.iloc[neighbours_indices_corrigidos]

,IBGE7,NOME,LATITUDE,LONGITUDE,TRANSPORT_COST,PREC_MED,RED_MED,TEMP_MED,VEL_MED,ALTITUDE,AREA_IRRIGADA_TOT,PRODUTO_MAIOR_VALOR,SOLO_PREDO,AREA_TOTAL
256,2926509,RIBEIRA DO AMPARO,-11.04210,-38.4242,23986.176670,778.182862,1034.162314,25.419589,2.172381,182.00,1545.743250,MELANCIA,RQo - Neossolos Quartzarenicos Orticos,61605.365659
257,2926608,RIBEIRA DO POMBAL,-10.83730,-38.5382,24140.726560,778.182862,1034.162314,25.419589,2.172381,182.00,509.000000,MELANCIA,RQo - Neossolos Quartzarenicos Orticos,75067.891352
55,2902658,BANZAE,-10.57880,-38.6212,24312.665944,600.502468,1250.912344,25.016424,2.710752,431.96,18.000000,MANDIOCA,RQo - Neossolos Quartzarenicos Orticos,15590.146022
230,2924009,PAULO AFONSO,-9.39830,-38.2216,24808.811543,551.553841,1437.777193,27.047011,2.518969,187.00,959.440042,UVA,RQo - Neossolos Quartzarenicos Orticos,73942.619394
817,2607000,INAJA,-8.90206,-37.8351,24941.784481,637.268386,1265.062885,25.636365,2.780842,434.23,2018.247340,TOMATE,RQo - Neossolos Quartzarenicos Orticos,100898.937234


In [21]:
df_classificador.loc[[160]]

,IBGE7,NOME,LATITUDE,LONGITUDE,TRANSPORT_COST,PREC_MED,RED_MED,TEMP_MED,VEL_MED,ALTITUDE,AREA_IRRIGADA_TOT,PRODUTO_MAIOR_VALOR,SOLO_PREDO,AREA_TOTAL
160,2916500,ITAPICURU,-11.3088,-38.2262,23766.055025,778.182862,1034.162314,25.419589,2.172381,182.0,644.0,MELAO,RQo - Neossolos Quartzarenicos Orticos,86920.869936


In [22]:
df_produto_agricola[df_produto_agricola["NOME"] == "ITAPICURU"]

,NOME,PRODUTO,AREA_PLANTADA,AREA_COLHIDA,REND_MEDIO,VALOR_PROD,IBGE7
3498,ITAPICURU,BANANA,20.000000,20.000000,13433.333333,569.666667,2916500
3499,ITAPICURU,CASTANHA DE CAJU,1295.666667,1295.666667,310.000000,1535.000000,2916500
3500,ITAPICURU,FEIJAO (EM GRAO),1006.666667,1006.666667,270.666667,1241.333333,2916500
3501,ITAPICURU,MANDIOCA,6266.666667,4133.333333,9092.666667,28589.000000,2916500
3502,ITAPICURU,MELANCIA,133.666667,133.666667,8848.333333,880.333333,2916500
3503,ITAPICURU,MELAO,223.666667,223.666667,21234.666667,4674.666667,2916500
3504,ITAPICURU,MILHO (EM GRAO),19333.333333,19333.333333,3689.000000,70163.666667,2916500
3505,ITAPICURU,SOJA (EM GRAO),51.666667,51.666667,2995.000000,309.000000,2916500


In [25]:
df_solo[df_solo["IBGE7"] == 2916500]

,IBGE7,SOLO,AREA_TOTAL
3023,2916500,LAd - Latossolos Amarelos Distroficos,18032.365183
3024,2916500,PVAd - Argissolos Vermelho-Amarelos Distroficos,46579.916148
3025,2916500,RQo - Neossolos Quartzarenicos Orticos,86920.869936
3026,2916500,SNo - Planossolos Natricos Orticos,15514.030117
